# Introduction

Ce TP continue le TP précédent. Nous allons reprendre d'ailleurs les mêmes données et commencer la mise en oeuvre d'un modèle de Markov pour la prédiction des étiquettes: 
* une observation est une phrase, représentée comme une séquence de variables aléatoires, une par mot de la phrase
* à cette observation est associée une séquence de variables aléatoires représentant les étiquettes, une par mot de la phrase également

On suppose que la séquence d'observation (une phrase) est générée par un modèle de Markov caché. Les variables cachées sont donc les étiquettes à inférer. Nous allons commencer par écrire une classe python pour représenter le HMM. Cette classe évoluera au fil des TPs. 

Pour cela le code de départ suivant est donné. Afin d'initialiser un HMM, nous devons connaitre : 
- l'ensemble des états (ou *state_list*), dans notre cas l'ensemble des étiquettes grammaticales;
- l'ensemble des observations (ou *observation_list*), dans notre cas l'ensemble des mots connus; tous les autres mots seront remplacés par l'élément spécial *UNK* qui fait partie de l'ensemble des observations. 

Enfin, en interne il est plus facile d'indexer les mots et et les états par des entiers. Ainsi à chaque éléments de respectivement l'ensemble des états et l'ensemble des observations, est associé un indice. Cela nous permet de tout traiter en "matricielle". 

# Interface avec les données et apprentissage supervisé

Ainsi pour initialiser un HMM, nous allons devoir lire les données (chose faîte lors du TP précédent): 
* écrire une fonction permettant d'initialiser le HMM à partir des données d'apprentissage
* écrire une fonction *apprentissage_supervisé* qui permet d'estimer les paramètres 

Dans un premier temps, nous limiterons, comme lors du TP précédent, le vocabulaire aux mots apparaissant 10 fois ou plus. Les autres mots sont tous remplacés par la même forme *unk*

Pour cela, le plan de travail peut être envisagé ainsi: 
* Lire les données puis générer un corpus de **train** (80%) puis de **test** (10%)
* écrire une fonction qui créer à partir des données d'apprentissage (**train**), tous les comptes nécessaires pour l'estimation supervisée des paramètres du HMM
* écrire 3 fonctions qui estimes les paramètres à partir des comptes, une fonction par distribution: observation, transition, état initial. 
* écrire une fonction qui reprend le tout et qui estime tous les paramètres du HMM


In [1]:
import numpy as np

from HMM import *
from toolbox import *

In [2]:
# Import data and separate datasets

import pickle
corpus = pickle.load( open( "../TP1/brown.save.p", "rb" ) )

train_set, valid_set, test_set = get_train_validation_test_sets(corpus, sets_ratio=(0.8, 0.1, 0.1))

In [3]:
# Initialize HMM

states, observations = get_states_observations(train_set)

hmm = HMM(states, observations)
# p_transitions = hmm.transition_proba
# p_emissions = hmm.observation_proba
# p_states_start = hmm.initial_state_proba
# hmm = HMM(states, observations, p_transitions, p_emissions, p_states_start)

HMM creating with: 
 * 12 states
 * 7609 observations


In [4]:
# train HMM on train set
hmm.train(train_set)

Training initial states probabilities... Done.
Training transitions probabilities given states... Done.
Training observations probabilities given states... Done.


In [5]:
# Try HMM prediction on one sample
SAMPLE = 2
observation_sequence = [token[0] for token in valid_set[SAMPLE]]
states_sequence = [token[1] for token in valid_set[SAMPLE]]

predicted_states_sequence = hmm.predict(observation_sequence)

print("Observation sequence      : {}".format(observation_sequence))
print("Real states sequence      : {}".format(states_sequence))
print("Predicted states sequence : {}".format(predicted_states_sequence))

Observation sequence      : ['Fire', 'Fighters', 'Local', '798', ',', 'which', 'is', 'sponsoring', 'the', 'toy', 'program', 'for', 'the', '12th', 'straight', 'year', ',', 'issued', 'a', 'call', 'for', 'San', 'Franciscans', 'to', 'turn', 'in', 'discarded', 'toys', ',', 'which', 'will', 'be', 'repaired', 'by', 'off-duty', 'firemen', '.']
Real states sequence      : ['NOUN', 'NOUN', 'NOUN', 'NUM', '.', 'DET', 'VERB', 'VERB', 'DET', 'NOUN', 'NOUN', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', '.', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'PRT', 'VERB', 'PRT', 'VERB', 'NOUN', '.', 'DET', 'VERB', 'VERB', 'VERB', 'ADP', 'ADJ', 'NOUN', '.']
Predicted states sequence : ['NOUN', 'VERB', 'ADJ', 'NOUN', '.', 'DET', 'VERB', 'VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', '.', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'VERB', 'PRT', 'VERB', 'ADP', 'ADJ', 'NOUN', '.', 'DET', 'VERB', 'VERB', 'NOUN', 'ADP', 'ADJ', 'NOUN', '.']


In [6]:
score_unk = hmm.score(test_set)
score_nounk = hmm.score(test_set, ignore_unk=True)
print("HMM score on test set :\n * {:.2f}% accuracy on words\n * {:.2f}% accuracy on full sentences".format(score_unk[0]*100, score_unk[1]*100))
print("HMM score on test set (ignoring UNK) :\n * {:.2f}% accuracy on words\n * {:.2f}% accuracy on full sentences".format(score_nounk[0]*100, score_nounk[1]*100))

HMM score on test set :
 * 93.56% accuracy on words
 * 36.82% accuracy on full sentences
HMM score on test set (ignoring UNK) :
 * 97.17% accuracy on words
 * 64.65% accuracy on full sentences



# Exercice : Algorithme de Viterbi

La question qui se pose est comment calculer la meilleure séquence d'étiquettes pour une phrase donnée connaissant les paramètres du HMM. Par meilleure, on entend la séquence d'étiquettes (ou d'états) la plus probable connaissant la séquence d'obervation. 

Proposer et implémenter un algorithme répondant à cette question. Pour vous aider à démarrer, cet algorithme s'appelle Viterbi et regardez cette vidéo https://www.youtube.com/watch?v=RwwfUICZLsA, pour comprendre comment il opère. 